# Rust 中的面向对象：Traits

## 前言

**面向对象的优点**

- 模块化：我们可以将一个大系统分解成可管理的组件，提供清晰的接口，并可以在隔离环境中进行测试。
- 封装性：将相关数据和方法组合到一个单独的“对象”中。代码隐藏：不需要暴露类中与用户交互无关的部分。
- 代码重用：想要根据输入文件使对象有所不同吗？只需向其构造函数添加一个参数，就可以得到两个不同的实现，但仅使用一个类！

**继承**可以重用代码

如图所示

<img src="./asset/traits/oop-bear.png" width="800" />

<img src="./asset/traits/inheritance.png" width="800" />

**继承**

优点
- 通过继承，我们能够在许多不同类型的对象之间使用相同的方法实现，这些对象通过父子关系聚集在一起。
- 子类继承所有方法和属性。（构造函数通常不计算在内，具体取决于语言）。它们可以选择覆盖父函数（绿色熊以不同方式咆哮）。
- 这是Java等语言中的重要概念（其中所有内容都继承自一个基本Object类）。

缺点
- 对于继承树，随着时间的推移维护和更改大型代码库

**其他**：
- 对象组合
    - 类A具有其他类类型的实例变量。
    - 例如：想要制作多种类型的填充动物。定义“毛皮”、“羽毛”、“爪子”、“嘴巴”等内容，并将它们组合成更复杂的填充动物。
    - 如果可能，比继承更好地实现了松散耦合。
- 多态性
    - 不同的底层类型/实现共享一个接口 
    - 例如：绿色熊从（基础）熊继承“咆哮”，但是绿色熊的“咆哮”实现方式不同。

## Traits

> 我们还能用什么方式分解？

如图所示：

<img src="./asset/traits/decompose-bear.png" width="800" />

In [ ]:
struct TeddyBear;
struct RedTeddyBear;
struct PurpleTeddyBear;
struct GreenTeddyBear;

impl TeddyBear {
	fn roar(&self) {
		println!("ROAR!!");
	}
}

impl RedTeddyBear {
	fn roar(&self) {
		println!("ROAR!!");
	}
	fn red_button_song(&self){
	    println!("RED SONG!");
	}
}

impl PurpleTeddyBear {
	fn roar(&self) {
		println!("ROAR!!");
	}
	fn purple_button_song(&self){
	    println!("PURPLE SONG!");
	}
}

impl GreenTeddyBear {
	fn roar(&self) {
		println!("ROAR!!");
	}
	fn green_button_song(&self){
	    println!("GREEN SONG!");
	}
}

fn main() {
    let bear = TeddyBear;
    let red_bear = RedTeddyBear;
    let green_bear = GreenTeddyBear;
    let purple_bear = PurpleTeddyBear;
    
    /* They all implement roar manually */
    bear.roar();
    red_bear.roar();
    green_bear.roar();
    purple_bear.roar();
    
    /* Some of them implement other functions */
    red_bear.red_button_song();
    green_bear.green_button_song();
    purple_bear.purple_button_song();
}

可以看出，所有的熊都存在`roar`方法。

可以将你想要的代码注入到一些类中（向它们注入一个 trait）

如图所示：

<img src="./asset/traits/inject-trait.png" width="800" />


In [ ]:
struct TeddyBear;
struct RedTeddyBear;
struct GreenTeddyBear;
struct PurpleTeddyBear;

// 定义一个 trait
trait Roar {
    fn roar(&self) {
        println!("ROAR!!");
    }
}

impl GreenTeddyBear {
	fn green_button_song(&self){
	    println!("GREEN SONG!");
	}
}

impl RedTeddyBear {
	fn red_button_song(&self){
	    println!("RED SONG!");
	}
}

impl PurpleTeddyBear {
	fn purple_button_song(&self){
	    println!("PURPLE SONG!");
	}
}

impl Roar for TeddyBear {}
impl Roar for RedTeddyBear {}
impl Roar for GreenTeddyBear {}
impl Roar for PurpleTeddyBear {}



fn main() {
    let bear = TeddyBear;
    let red_bear = RedTeddyBear;
    let green_bear = GreenTeddyBear;
    let purple_bear = PurpleTeddyBear;
    
    bear.roar();
    red_bear.roar();
    green_bear.roar();
    purple_bear.roar();
    
    red_bear.red_button_song();
    green_bear.green_button_song();
    purple_bear.purple_button_song();
}

### 制作第一个 `trait`

trait的背景：可以查看此[链接](https://blog.rust-lang.org/2015/05/11/traits.html)

- 通过 `trait`，可以编写可注入到任何现有结构中的代码。（从 `TeddyBear` 类型到 `i32` 类型！）此代码可以引用 _自身_，因此该代码可能依赖于实例。
- `Trait` 方法不需要完全定义 -- 可以定义一个函数，该函数在为类型实现`Trait` 时必须实现。（类似于Java接口）
- `Trait` 可以指定函数/数据实例 __应具有__ 的内容，而不仅仅是从另一个“父级”获取许多内容。
- 没有更深层次的继承层次结构了。只需思考：“这种类型是否实现了这个trait？” 

### 大型标准 Rust 特性

- __`Copy`__: 使用赋值（=）时，将创建实例的新副本，而不是移动所有权。
- __`Clone`__: 在方法上调用 `.clone()` 函数时，将返回实例的新副本。
- __`Drop`__: 定义一种释放实例内存的方式 -- 当实例到达作用域末尾时调用。
- __`Display`__: 定义了一种格式化类型并显示它的方式（由 `println!`使用）。
- __`Debug`__: 类似于 Display，但不适合用户界面（适用于您调试类型！）
- __`Eq`__: 定义了两个相同类型对象之间确定等价关系的相等性比较方式。
- __`PartialOrd`__: 定义了比较实例（小于、大于、小于或等于等）的方法。

实现一个标准的 `Trait`

```rs
struct Point {
    x: u32,
    y: u32,
}

fn main() {
    let pt = Point{x: 3, y: 2};
    let pt2 = pt.clone();
}
```

代码不会被编译，因为 `clone()` 函数没有被定义

让我们注入 `Clone`


In [3]:
use std::fmt;
struct Point {
    x: u32,
    y: u32,
}

impl Clone for Point {
    fn clone(&self) -> Point {
        let new_pt = Point {x: self.x, y: self.y};
        new_pt
    }
}

fn main() {
  let pt = Point {x:3, y:2};
  let pt2 = pt.clone();
  
  if pt.x == pt2.x && pt.y == pt2.y {
      println!("We have made two points that are the same!");
  }
}

// main()

We have made two points that are the same!


()

- 可以将任何 `trait` 实现到任何结构体中（就像我们对 Point 实现 Clone 特性一样），只要它们是兼容的（`Drop` 与 `Copy` 不兼容）。
- 你可以使用 [Rust 文档](https://doc.rust-lang.org/std/clone/trait.Clone.html) 来查看需要实现哪些函数以及它们的参数类型。
- 你可以使用 #[derive(x,y,z..)] 来派生特性。如果你的结构体满足某些规则（由文档给出），Rust 编译器会尝试为你实现这些特性。例如：如果结构体中所有成员都已经实现了 Clone，那么你就可以派生 Clone。

### 练习

💡 想法：定义代表不同种类植物的类型。
- 它们都将有自定义实现的特征，如“浇水”和“需要浇水”。
- 还希望派生（derive）一些有用的特征，例如打印出植物当前状态以进行调试。

In [ ]:
pub struct SensitivePlant {
    last_poked: DateTime<Local>,
    last_watered: DateTime<Local>,
}

impl SensitivePlant {
    pub fn new() -> SensitivePlant {
        SensitivePlant {last_poked: Local::now(), last_watered: Local::now()}
    }

    pub fn poke(&mut self) {
        self.last_poked = Local::now();
    }

    pub fn is_open(&self) -> bool {
        (Local::now() - self.last_poked).num_seconds() > 2
    }

    pub fn last_watered(&self) -> DateTime<Local> {
        self.last_watered
    }

    pub fn needs_watering(&self) -> bool {
        (Local::now() - self.last_watered) > 3
    }

    pub fn water(&mut self) {
        self.last_watered = Local::now();
    }
}

pub struct StringOfTurtles {
    num_turtles: usize,
    last_watered: DateTime<Local>,
}

impl StringOfTurtles {
    pub fn new() -> StringOfTurtles {
        StringOfTurtles{ num_turtles: 20, last_watered: Local::now() }
    }

    pub fn turtles(&self) -> usize {
        self.num_turtles
    }

    pub fn last_watered(self) -> DateTime<Local> {
        self.last_watered
    }

    pub fn needs_watering(&self) -> bool {
        (Local::now() - self.last_watered).num_days() > 20
    }

    pub fn water(&mut self) {
        self.last_watered = Local::now();
    }
}

1. derive Debug

- 在 Rust 中打印的两种方式：
    - `Display`：清晰、简单的表示方法，使用 `println!("{}", object)`调用；
    - `Debug`: 更详细的表示方法，用于调试，使用 `printin!("{:?}", object)` 调用。
- 如果所有成员都是 Debug，则可以 `#derive Debug`
    - （注意：usize和DateTime是Debug）。我们该如何在这里实现呢？ 

```rs
#[derive(Debug)]
```

2.  functions -> traits

观察上面的代码，两者有时候都需要浇水以及可以浇水。。。。，这听起来是一个很好的 trait 候选，如何实现呢？

```rs
pub trait NeedsWater {
    // Define funtion signatures here.
    // what function should a plane that "drinks" water implement?
}


impl /* Trait name*/ NeedsWater for /* struct type */ StringOfTurtles {
    // what's the custom behavior for the specific type for the function that a "NeedsWater" plant is required to implement?
}
```

In [ ]:
use chrono::{DateTime, Local};

#[derive(Debug)]
pub struct SensitivePlant {
    last_poked: DateTime<Local>,
    last_watered: DateTime<Local>,
}

#[derive(Debug)]
pub struct StringOfTurtles {
    num_turtles: usize,
    last_watered: DateTime<Local>,
}

pub trait NeedsWater {
    fn needs_watering(&self) -> bool{}

    fn last_watered(&self) -> DateTime<Local> {
        self.last_watered
    }
    
    fn water(&mut self) {
        self.last_watered = Local::now();
    }
}

impl NeedsWater for SensitivePlant {
    fn needs_watering(&self) -> bool {
        (Local::now() - self.last_watered) > 3
    }
}

impl NeedsWater for StringOfTurtles {
    fn needs_watering(&self) -> bool {
        (Local::now() - self.last_watered).num_days() > 20
    }
}

impl SensitivePlant {
    pub fn new() -> SensitivePlant {
        SensitivePlant {last_poked: Local::now(), last_watered: Local::now()}
    }

    pub fn poke(&mut self) {
        self.last_poked = Local::now();
    }

    pub fn is_open(&self) -> bool {
        (Local::now() - self.last_poked).num_seconds() > 2
    }
}

impl StringOfTurtles {
    pub fn new() -> StringOfTurtles {
        StringOfTurtles{ num_turtles: 20, last_watered: Local::now() }
    }

    pub fn turtles(&self) -> usize {
        self.num_turtles
    }
}